In [1]:
import tensorflow as tf 
import numpy as np
from modules.training.cme_modeling import ModelBuilder as MB

C:\Users\the_3\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\the_3\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If 

In [2]:
from modules.training.phase_manager import TrainingPhaseManager

# instantiate the model_builder
mb = MB()
pm = TrainingPhaseManager()  # Training phase manager

In [5]:
# y_true = tf.constant([[-1.0],
#                       [ 3],
#                       [ 1.0]])
# 
# z_pred = tf.constant([[-1.0,  1.0],
#                       [ 0.5,  0.0],
#                       [ 1.0, -1.0],
#                       ])

y_true = tf.constant([[1.0],
                      [2.0],
                      [3.0]])

z_pred = tf.constant([[1.0, 1.0],
                      [ 2.0, 2.0],
                      [3.0, 3.0],
                      ])


In [6]:
loss = mb.pdc_loss_vec(y_true, z_pred, pm)
print(f"PDC Loss: {loss.numpy()}")

PDC Loss: 0.0


In [5]:
loss = mb.pdc_loss_vec_mem(y_true, z_pred, pm)
print(f"PDC Loss: {loss.numpy()}")

PDC Loss: 1.2289680242538452
